In [1]:
import os
import cv2
import glob
import h5py
import imgaug as aug
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import imgaug.augmenters as iaa
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from pathlib import Path
from skimage.io import imread
from skimage.transform import resize
from keras.models import Sequential, Model, load_model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten,Activation
from keras.models import Sequential
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from PIL import Image
import PIL
import scipy as sp 
import scipy.ndimage as spi
%matplotlib inline

color = sns.color_palette()
%matplotlib inline
%config InlineBackend.figure_format="svg"

import tensorflow as tf

Using TensorFlow backend.


In [2]:
# Set the seed for hash based operations in python
os.environ['PYTHONHASHSEED'] = '0'

seed=1234

# Set the numpy seed
np.random.seed(seed)

# Set the random seed in tensorflow at graph level
tf.set_random_seed(seed)

# Make the augmentation sequence deterministic
aug.seed(seed)

In [3]:
training_data = Path('training') 
validation_data = Path('validation') 
labels_path = Path('monkey_labels.txt')

In [4]:
monkey_labels = []

# Read the file
lines = labels_path.read_text().strip().splitlines()[1:]

In [5]:
lines

['n0   , alouatta_palliata\t , mantled_howler                , 131          , 26',
 'n1   , erythrocebus_patas\t , patas_monkey                  , 139          , 28',
 'n2   , cacajao_calvus\t     , bald_uakari                   , 137          , 27',
 'n3   , macaca_fuscata\t     , japanese_macaque              , 152          , 30',
 'n4   , cebuella_pygmea\t     , pygmy_marmoset                , 131          , 26',
 'n5   , cebus_capucinus\t     , white_headed_capuchin         , 141          , 28',
 'n6   , mico_argentatus\t     , silvery_marmoset              , 132          , 26',
 'n7   , saimiri_sciureus\t     , common_squirrel_monkey        , 142          , 28',
 'n8   , aotus_nigriceps\t     , black_headed_night_monkey     , 133          , 27',
 'n9   , trachypithecus_johnii , nilgiri_langur                , 132          , 26']

In [6]:
for line in lines:
    line = line.split(',')
    line = [x.strip(' \n\t\r') for x in line]
    line[3], line[4] = int(line[3]), int(line[4])
    line = tuple(line)
    monkey_labels.append(line)

In [7]:
monkey_labels = pd.DataFrame(monkey_labels, columns=['Label', 'Latin Name', 'Common Name','Train Images', 'Validation Images'], index=None)


In [8]:
monkey_labels

,Label,Latin Name,Common Name,Train Images,Validation Images
0,n0,alouatta_palliata,mantled_howler,131,26
1,n1,erythrocebus_patas,patas_monkey,139,28
2,n2,cacajao_calvus,bald_uakari,137,27
3,n3,macaca_fuscata,japanese_macaque,152,30
4,n4,cebuella_pygmea,pygmy_marmoset,131,26
5,n5,cebus_capucinus,white_headed_capuchin,141,28
6,n6,mico_argentatus,silvery_marmoset,132,26
7,n7,saimiri_sciureus,common_squirrel_monkey,142,28
8,n8,aotus_nigriceps,black_headed_night_monkey,133,27
9,n9,trachypithecus_johnii,nilgiri_langur,132,26


In [9]:
labels=pd.DataFrame()
labels["id"] = monkey_labels["Label"].str.strip()
labels["name"] = monkey_labels["Common Name"].str.strip()
labels

,id,name
0,n0,mantled_howler
1,n1,patas_monkey
2,n2,bald_uakari
3,n3,japanese_macaque
4,n4,pygmy_marmoset
5,n5,white_headed_capuchin
6,n6,silvery_marmoset
7,n7,common_squirrel_monkey
8,n8,black_headed_night_monkey
9,n9,nilgiri_langur


In [10]:
# Create a dictionary to map the labels to integers
m_id= labels["id"]
m_id

0    n0
1    n1
2    n2
3    n3
4    n4
5    n5
6    n6
7    n7
8    n8
9    n9
Name: id, dtype: object

In [11]:
entries = os.listdir(training_data)
for entry in entries:
    for f in glob.glob(os.path.join(os.path.join(training_data,entry),"*.jpg")):
        W = 250.
        oriimg = cv2.imread(f,cv2.IMREAD_COLOR)
        depth = oriimg.shape
        imgScale = W/250
        newX,newY = 250*imgScale, 250*imgScale
        newimg = cv2.resize(oriimg,(int(newX),int(newY)))
        cv2.imwrite(f,newimg)        

entries2 = os.listdir(validation_data)

for entry in entries2:
    for f in glob.glob(os.path.join(os.path.join(validation_data,entry),"*.jpg")):
        W = 250.
        oriimg = cv2.imread(f,cv2.IMREAD_COLOR)
        depth = oriimg.shape
        imgScale = W/250
        newX,newY = 250*imgScale, 250*imgScale
        newimg = cv2.resize(oriimg,(int(newX),int(newY)))
        cv2.imwrite(f,newimg)
        
        

In [12]:
image_width=250
image_height=250
batch_size= 16


In [13]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

In [14]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [15]:
train_generator = train_datagen.flow_from_directory(training_data, 
                                                    target_size=(image_width, image_height), 
                                                    batch_size = batch_size, 
                                                    shuffle=True, # By shuffling the images we add some randomness and prevent overfitting
                                                    class_mode="categorical")

Found 1096 images belonging to 10 classes.


In [16]:
validation_generator = validation_datagen.flow_from_directory(validation_data, 
                                                    target_size=(image_width, image_height), 
                                                    batch_size = batch_size, 
                                                    shuffle=True,
                                                    class_mode="categorical")

Found 272 images belonging to 10 classes.


In [17]:
training_samples = 1097
validation_samples = 272
total_steps = training_samples // batch_size

In [18]:
model = VGG16(weights='imagenet', include_top=False, input_shape=(image_width, image_height, 3), pooling="max")

Instructions for updating:
Colocations handled automatically by placer.


In [19]:
for layer in model.layers[:-5]:
        layer.trainable = False

In [20]:
for layer in model.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f3e83734240> False
<keras.layers.convolutional.Conv2D object at 0x7f3e83734cf8> False
<keras.layers.convolutional.Conv2D object at 0x7f3e837343c8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f3e83757898> False
<keras.layers.convolutional.Conv2D object at 0x7f3e83757828> False
<keras.layers.convolutional.Conv2D object at 0x7f3e81ef8a20> False
<keras.layers.pooling.MaxPooling2D object at 0x7f3e81ea8278> False
<keras.layers.convolutional.Conv2D object at 0x7f3e81f11e80> False
<keras.layers.convolutional.Conv2D object at 0x7f3e81ec4da0> False
<keras.layers.convolutional.Conv2D object at 0x7f3e81e755f8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f3e81e91d68> False
<keras.layers.convolutional.Conv2D object at 0x7f3e81e91a20> False
<keras.layers.convolutional.Conv2D object at 0x7f3e81e408d0> False
<keras.layers.convolutional.Conv2D object at 0x7f3e81e5d8d0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f3e81e10898>

In [21]:
transfer_model = Sequential()
for layer in model.layers:
    transfer_model.add(layer)
transfer_model.add(Dense(512, activation="relu"))
transfer_model.add(Dropout(0.5))
transfer_model.add(Dense(10, activation="softmax"))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [22]:
from keras import optimizers
adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.00001)

transfer_model.compile(loss="categorical_crossentropy",
                      optimizer=adam,
                      metrics=["accuracy"])

In [23]:
model_history = transfer_model.fit_generator(train_generator, steps_per_epoch=training_samples // batch_size,
                                            epochs=21,
                                            validation_data=validation_generator,
                                            validation_steps=validation_samples // batch_size)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/21
68/68 [==============================] - 855s 13s/step - loss: 2.2227 - acc: 0.2188 - val_loss: 1.4943 - val_acc: 0.5331
Epoch 2/21
68/68 [==============================] - 717s 11s/step - loss: 1.2957 - acc: 0.5653 - val_loss: 0.6515 - val_acc: 0.8051
Epoch 3/21
68/68 [==============================] - 720s 11s/step - loss: 0.8255 - acc: 0.7335 - val_loss: 0.4970 - val_acc: 0.8419
Epoch 4/21
68/68 [==============================] - 745s 11s/step - loss: 0.5501 - acc: 0.8373 - val_loss: 0.4070 - val_acc: 0.8897
Epoch 5/21
68/68 [==============================] - 738s 11s/step - loss: 0.4357 - acc: 0.8419 - val_loss: 0.4453 - val_acc: 0.8566
Epoch 6/21
68/68 [==============================] - 836s 12s/step - loss: 0.3574 - acc: 0.8851 - val_loss: 0.3507 - val_acc: 0.8897
Epoch 7/21
68/68 [==============================] - 944s 14s/step - loss: 0.3152 -